In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 2.8MB/s 
     |████████████████████████████████| 2.3MB 5.8MB/s 
     |████████████████████████████████| 1.2MB 44.6MB/s 
     |████████████████████████████████| 3.3MB 48.2MB/s 
     |████████████████████████████████| 901kB 47.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=28ae629cb8418465e4610cfca22d38315793117ab0b54d9c6873662ad51bdefa
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [ ]:
import pickle
from sentence_transformers import SentenceTransformer, util
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import numpy as np
import math
import torch
from string import punctuation
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Find closest BabyAI instruction for each user sentence

In [ ]:
# Our tool (i.e. sentences for each individual task, with the demonstration constraint)
all_sentences = [pickle.load(open('drive/MyDrive/babyai_sentences' + str(i) + '.pkl', 'rb')) for i in range(17)]
len(all_sentences)

17

In [ ]:
# No-demonstration baseline (sentences across all tasks)
ah = [sentence for l in all_sentences for sentence in l]
len(ah)

195401

In [ ]:
model = SentenceTransformer('stsb-distilbert-base')

In [ ]:
len(all_sentences[0]) + len(all_sentences[1]) + len(all_sentences[2]) + len(all_sentences[3]) + len(all_sentences[4]) + len(all_sentences[5]) + len(all_sentences[6]) + len(all_sentences[7]) + len(all_sentences[8]) + len(all_sentences[9]) + len(all_sentences[10]) + len(all_sentences[11]) + len(all_sentences[12]) + len(all_sentences[13]) + len(all_sentences[14]) + len(all_sentences[15]) + len(all_sentences[16])

195401

In [ ]:
# Embeddings for our tool
embeddings = [model.encode(all_sentences[i], convert_to_tensor=True) for i in range(len(all_sentences))]

In [ ]:
# Embeddings for no demo
ah2 = model.encode(ah, convert_to_tensor=True)
len(ah2)

195401

In [ ]:
len(embeddings)

17

In [ ]:
def remove_punctuation(text):
    for punct in punctuation:
        text = text.replace(punct, '')
    return text.lower()

## No training

In [ ]:
v0_csv = pd.read_csv('drive/MyDrive/Baby AI Instructions (V0).csv')
instructions_v0 = [v0_csv[column].apply(remove_punctuation).values for column in ['Q5', 'Q6', 'Q7', 'Q8', 'Q10', 'Q11', 'Q12', 'Q13', 'Q18', 'Q19', 'Q20', 'Q21', 'Q23', 'Q24', 'Q25', 'Q26', 'Q27']]
gold_sentences = ['go to the green ball', 'put the red ball next to the yellow box', 'go to the red box', 'go to the purple key',
                  'put the green box next to the green ball', 'go to the grey ball', 'go to the red ball', 'go to the grey key',
                  'pick up a key and put the green key next to the grey box',
                  'pick up a green key , then put the yellow box next to the grey ball',
                  'open the yellow door after you open a purple door', 'go to a door after you pick up the yellow box',
                  'go to the purple ball after you open a door', 'put a box next to the green key and open a green door',
                  'put the blue key next to the yellow box and put the red box next to a blue door',
                  'go to a yellow door and put a green box next to a green door', 'pick up a box , then pick up a ball']
output = {'Task 1 i': ['gold instruction', gold_sentences[0], '', 'original instruction'], 'Task 1 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 1 s': ['', '', '', 'cosine score'],
          'Task 2 i': ['gold instruction', gold_sentences[1], '', 'original instruction'], 'Task 2 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 2 s': ['', '', '', 'cosine score'],
          'Task 3 i': ['gold instruction', gold_sentences[2], '', 'original instruction'], 'Task 3 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 3 s': ['', '', '', 'cosine score'],
          'Task 4 i': ['gold instruction', gold_sentences[3], '', 'original instruction'], 'Task 4 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 4 s': ['', '', '', 'cosine score'],
          'Task 6 i': ['gold instruction', gold_sentences[4], '', 'original instruction'], 'Task 6 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 6 s': ['', '', '', 'cosine score'],
          'Task 7 i': ['gold instruction', gold_sentences[5], '', 'original instruction'], 'Task 7 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 7 s': ['', '', '', 'cosine score'],
          'Task 8 i': ['gold instruction', gold_sentences[6], '', 'original instruction'], 'Task 8 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 8 s': ['', '', '', 'cosine score'],
          'Task 9 i': ['gold instruction', gold_sentences[7], '', 'original instruction'], 'Task 9 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 9 s': ['', '', '', 'cosine score'],
          'Task 11 i': ['gold instruction', gold_sentences[8], '', 'original instruction'], 'Task 11 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 11 s': ['', '', '', 'cosine score'],
          'Task 12 i': ['gold instruction', gold_sentences[9], '', 'original instruction'], 'Task 12 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 12 s': ['', '', '', 'cosine score'],
          'Task 13 i': ['gold instruction', gold_sentences[10], '', 'original instruction'], 'Task 13 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 13 s': ['', '', '', 'cosine score'],
          'Task 14 i': ['gold instruction', gold_sentences[11], '', 'original instruction'], 'Task 14 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 14 s': ['', '', '', 'cosine score'],
          'Task 16 i': ['gold instruction', gold_sentences[12], '', 'original instruction'], 'Task 16 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 16 s': ['', '', '', 'cosine score'],
          'Task 17 i': ['gold instruction', gold_sentences[13], '', 'original instruction'], 'Task 17 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 17 s': ['', '', '', 'cosine score'],
          'Task 18 i': ['gold instruction', gold_sentences[14], '', 'original instruction'], 'Task 18 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 18 s': ['', '', '', 'cosine score'],
          'Task 19 i': ['gold instruction', gold_sentences[15], '', 'original instruction'], 'Task 19 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 19 s': ['', '', '', 'cosine score'],
          'Task 20 i': ['gold instruction', gold_sentences[16], '', 'original instruction'], 'Task 20 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 20 s': ['', '', '', 'cosine score']}
for i, task in enumerate(instructions_v0):
  matches = []
  scores = []
  correct = 0
  for instr in task:
    embedding = model.encode(instr, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding, ah2)
    max = torch.max(cosine_scores[0])
    closest = ah[cosine_scores.argmax(1)]
    print(instr)
    matches.append(closest)
    scores.append(max.item())
    print(max.item())
    print(closest)
    gold_tokens = ' '.join([token for token in nltk.word_tokenize(gold_sentences[i]) if token not in ['a', 'the']])
    match_tokens = ' '.join([token for token in nltk.word_tokenize(closest) if token not in ['a', 'the']])
    if gold_tokens == match_tokens:
      print('yo wassup')
      correct += 1
  if i < 4:
    output['Task ' + str(i+1) + ' i'].extend(task)
    output['Task ' + str(i+1) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+1) + ' m'].extend(matches)
    output['Task ' + str(i+1) + ' s'].extend(scores)
  elif i < 8:
    output['Task ' + str(i+2) + ' i'].extend(task)
    output['Task ' + str(i+2) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+2) + ' m'].extend(matches)
    output['Task ' + str(i+2) + ' s'].extend(scores)
  elif i < 12:
    output['Task ' + str(i+3) + ' i'].extend(task)
    output['Task ' + str(i+3) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+3) + ' m'].extend(matches)
    output['Task ' + str(i+3) + ' s'].extend(scores)
  else:
    output['Task ' + str(i+4) + ' i'].extend(task)
    output['Task ' + str(i+4) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+4) + ' m'].extend(matches)
    output['Task ' + str(i+4) + ' s'].extend(scores)

go to the green circle
0.7680684924125671
go to the green box
i believe robot needs to collect keys in order to have access to the green kinda door looking object 
0.62285315990448
go to a green box after you pick up the green key, then go to a grey door
go to the green ball and stop in front of it
0.8150744438171387
go to a green ball and go to the green ball
a red arrow pointer moving towards greeen point
0.4990313649177551
go to a red ball
go to the green circle
0.7680684924125671
go to the green box
go stand in front of the green dot
0.7573397755622864
go to the green box
go to the green circle in the upper right corner
0.6712142825126648
go to the green key and go to the green key
go to the green ball
1.0
go to the green ball
yo wassup
move to the green ball and face it
0.7751110792160034
go to the green ball
yo wassup
find the green ball 
0.8783243894577026
go to the green ball
yo wassup
go to the green circle
0.7680684924125671
go to the green box
 find the green dot
0.800766766

In [ ]:
df0 = pd.DataFrame.from_dict(output)
df0.to_csv('drive/MyDrive/preliminary_results_v0-all.csv', index=False)

## Training

In [ ]:
v1_csv = pd.read_csv('drive/MyDrive/Baby AI Instructions (V1).csv')
instructions_v1 = [v1_csv[column].apply(remove_punctuation).values for column in ['Q5', 'Q6', 'Q7', 'Q8', 'Q10', 'Q11', 'Q12', 'Q13', 'Q33', 'Q34', 'Q35', 'Q36', 'Q38', 'Q39', 'Q40', 'Q41', 'Q42']]
gold_sentences = ['go to the green ball', 'put the red ball next to the yellow box', 'go to the red box', 'go to the purple key',
                  'put the green box next to the green ball', 'go to the grey ball', 'go to the red ball', 'go to the grey key',
                  'pick up a key and put the green key next to the grey box',
                  'pick up a green key , then put the yellow box next to the grey ball',
                  'open the yellow door after you open a purple door', 'go to a door after you pick up the yellow box',
                  'go to the purple ball after you open a door', 'put a box next to the green key and open a green door',
                  'put the blue key next to the yellow box and put the red box next to a blue door',
                  'go to a yellow door and put a green box next to a green door', 'pick up a box , then pick up a ball']
output = {'Task 1 i': ['gold instruction', gold_sentences[0], '', 'original instruction'], 'Task 1 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 1 s': ['', '', '', 'cosine score'],
          'Task 2 i': ['gold instruction', gold_sentences[1], '', 'original instruction'], 'Task 2 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 2 s': ['', '', '', 'cosine score'],
          'Task 3 i': ['gold instruction', gold_sentences[2], '', 'original instruction'], 'Task 3 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 3 s': ['', '', '', 'cosine score'],
          'Task 4 i': ['gold instruction', gold_sentences[3], '', 'original instruction'], 'Task 4 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 4 s': ['', '', '', 'cosine score'],
          'Task 6 i': ['gold instruction', gold_sentences[4], '', 'original instruction'], 'Task 6 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 6 s': ['', '', '', 'cosine score'],
          'Task 7 i': ['gold instruction', gold_sentences[5], '', 'original instruction'], 'Task 7 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 7 s': ['', '', '', 'cosine score'],
          'Task 8 i': ['gold instruction', gold_sentences[6], '', 'original instruction'], 'Task 8 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 8 s': ['', '', '', 'cosine score'],
          'Task 9 i': ['gold instruction', gold_sentences[7], '', 'original instruction'], 'Task 9 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 9 s': ['', '', '', 'cosine score'],
          'Task 11 i': ['gold instruction', gold_sentences[8], '', 'original instruction'], 'Task 11 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 11 s': ['', '', '', 'cosine score'],
          'Task 12 i': ['gold instruction', gold_sentences[9], '', 'original instruction'], 'Task 12 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 12 s': ['', '', '', 'cosine score'],
          'Task 13 i': ['gold instruction', gold_sentences[10], '', 'original instruction'], 'Task 13 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 13 s': ['', '', '', 'cosine score'],
          'Task 14 i': ['gold instruction', gold_sentences[11], '', 'original instruction'], 'Task 14 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 14 s': ['', '', '', 'cosine score'],
          'Task 16 i': ['gold instruction', gold_sentences[12], '', 'original instruction'], 'Task 16 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 16 s': ['', '', '', 'cosine score'],
          'Task 17 i': ['gold instruction', gold_sentences[13], '', 'original instruction'], 'Task 17 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 17 s': ['', '', '', 'cosine score'],
          'Task 18 i': ['gold instruction', gold_sentences[14], '', 'original instruction'], 'Task 18 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 18 s': ['', '', '', 'cosine score'],
          'Task 19 i': ['gold instruction', gold_sentences[15], '', 'original instruction'], 'Task 19 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 19 s': ['', '', '', 'cosine score'],
          'Task 20 i': ['gold instruction', gold_sentences[16], '', 'original instruction'], 'Task 20 m': ['overall matching accuracy', 0, '', 'closest match'], 'Task 20 s': ['', '', '', 'cosine score']}
for i, task in enumerate(instructions_v1):
  matches = []
  scores = []
  correct = 0
  for instr in task:
    embedding = model.encode(instr, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embedding, ah2)
    max = torch.max(cosine_scores[0])
    closest = ah[cosine_scores.argmax(1)]
    print(instr)
    matches.append(closest)
    scores.append(max.item())
    print(max.item())
    print(closest)
    gold_tokens = ' '.join([token for token in nltk.word_tokenize(gold_sentences[i]) if token not in ['a', 'the']])
    match_tokens = ' '.join([token for token in nltk.word_tokenize(closest) if token not in ['a', 'the']])
    if gold_tokens == match_tokens:
      print('yo wassup')
      correct += 1
  if i < 4:
    output['Task ' + str(i+1) + ' i'].extend(task)
    output['Task ' + str(i+1) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+1) + ' m'].extend(matches)
    output['Task ' + str(i+1) + ' s'].extend(scores)
  elif i < 8:
    output['Task ' + str(i+2) + ' i'].extend(task)
    output['Task ' + str(i+2) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+2) + ' m'].extend(matches)
    output['Task ' + str(i+2) + ' s'].extend(scores)
  elif i < 12:
    output['Task ' + str(i+3) + ' i'].extend(task)
    output['Task ' + str(i+3) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+3) + ' m'].extend(matches)
    output['Task ' + str(i+3) + ' s'].extend(scores)
  else:
    output['Task ' + str(i+4) + ' i'].extend(task)
    output['Task ' + str(i+4) + ' m'][1] = correct / len(task)
    output['Task ' + str(i+4) + ' m'].extend(matches)
    output['Task ' + str(i+4) + ' s'].extend(scores)

go to the green ball
1.0
go to the green ball
yo wassup
go to the green ball
1.0
go to the green ball
yo wassup
go to the green ball
1.0
go to the green ball
yo wassup
go to the green ball
1.0
go to the green ball
yo wassup
go to the green dot
0.8267195224761963
go to the green box
go to the green ball
1.0
go to the green ball
yo wassup
go to a blue ball then go to a green ball
0.9883865714073181
go to a green ball, then go to a blue ball
go to the green ball
1.0
go to the green ball
yo wassup
go to green ball
0.9793695211410522
go to the green ball
yo wassup
go to the green ball
1.0
go to the green ball
yo wassup
go to the blue ball then the yellow ball
0.8593615889549255
go to a yellow ball, then go to a yellow ball
go the green ball
0.9358314871788025
go to the green ball
yo wassup
go to the green ball
1.0
go to the green ball
yo wassup
go to the green ball
1.0
go to the green ball
yo wassup
go to the green ball
1.0
go to the green ball
yo wassup
pick green ball
0.8685245513916016
g

In [ ]:
df1 = pd.DataFrame.from_dict(output)
df1.to_csv('drive/MyDrive/preliminary_results_v1-all.csv', index=False)

## GPT-2

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
def score(sentence):
  tokenize_input = tokenizer.tokenize(sentence)
  tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
  loss=model(tensor_input, labels=tensor_input)[0]
  return math.exp(loss)

In [ ]:
csv = pd.read_csv('drive/MyDrive/preliminary_results.csv')
all_instructions = [csv['Task ' + str(i) + ' i'] for i in [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 20]]
all_matches = [csv['Task ' + str(i) + ' m'] for i in [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 20]]
all_matches_set = [set(i) for i in all_matches]
len(all_instructions[0])

127

In [ ]:
len(all_matches)

17

In [ ]:
most_likely = []
for i in range(len(all_matches_set)):
  curr_min = 10000000000000000000
  curr_likely = ''
  for text in all_matches_set[i]:         
      curr_score = score(text)
      if curr_score < curr_min:
        curr_min = curr_score
        curr_likely = text
  most_likely.append(text)

In [ ]:
most_likely

['go to a blue ball',
 'go to the red key, then go to a red ball, then put a red ball next to the yellow box',
 'open a blue door and open a purple door and go to a red box',
 'go to the blue box, then go to the yellow ball and go to a purple key',
 'pick up the green box, then go to a blue key, then put a green box next to a green ball',
 'go to the grey ball',
 'go to the grey door and open a grey door and go to the red ball',
 'open a grey door',
 'go to a blue box',
 'go to a green key and pick up the yellow box, then go to the purple key',
 'open the purple door, then go to the green ball, then go to the red box',
 'pick up a yellow box and go to a red door',
 'go to the red box and go to a purple key',
 'pick up the grey box and put the grey box next to a green key',
 'open the blue door, then pick up the blue key and go to a red box',
 'go to the yellow door and go to a green box',
 'open the red door and go to the red key and open the purple door']

In [ ]:
most_likely = ['go to a blue ball',
 'go to the red key, then go to a red ball, then put a red ball next to the yellow box',
 'open a blue door and open a purple door and go to a red box',
 'go to the blue box, then go to the yellow ball and go to a purple key',
 'pick up the green box, then go to a blue key, then put a green box next to a green ball',
 'go to the grey ball',
 'go to the grey door and open a grey door and go to the red ball',
 'open a grey door',
 'go to a blue box',
 'go to a green key and pick up the yellow box, then go to the purple key',
 'open the purple door, then go to the green ball, then go to the red box',
 'pick up a yellow box and go to a red door',
 'go to the red box and go to a purple key',
 'pick up the grey box and put the grey box next to a green key',
 'open the blue door, then pick up the blue key and go to a red box',
 'go to the yellow door and go to a green box',
 'open the red door and go to the red key and open the purple door']

In [ ]:
csv_all = pd.read_csv('drive/MyDrive/preliminary_results-all.csv')
all_matches_all = [csv_all['Task ' + str(i) + ' m'] for i in [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 20]]
len(all_matches_all[0])

127

# Evaluate results

In [ ]:
results_csv = pd.read_csv('drive/MyDrive/Baby AI Evaluation Take All.csv')
results_csv

,QID32,QID33,QID34,QID35,QID36,QID37,QID38,QID39,QID40,QID41,QID42,QID43,QID44,QID45,QID46,QID47,QID48,QID49,QID50,QID51,QID52,QID53,QID54,QID55,QID56,QID57,QID58,QID59,QID60,QID61,QID62,QID63,QID64,QID65,QID66,QID67,QID68,QID69,QID70,QID71,...,QID1472,QID1473,QID1474,QID1475,QID1476,QID1477,QID1478,QID1479,QID1480,QID1481,QID1482,QID1483,QID1484,QID1485,QID1486,QID1487,QID1488,QID1489,QID1490,QID1491,QID1492,QID1493,QID1494,QID1495,QID1496,QID1497,QID1498,QID1499,QID1500,QID1501,QID1502,QID1503,QID1504,QID1505,QID1506,QID1507,QID1508,QID1509,QID1510,Random ID
0,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a video illustrating thi...,Task 8\n\n\n\nHere is a video illustrating thi...,Task 9\n\n\n\nHere is a video illustrating thi...,Task 10\n\n\n\nHere is a video illustrating th...,Task 11\n\n\n\nHere is a video illustrating th...,Task 12\n\n\n\nHere is a video illustrating th...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a video illustrating thi...,Task 8\n\n\n\nHere is a video illustrating thi...,Task 9\n\n\n\nHere is a video illustrating thi...,Task 10\n\n\n\nHere is a video illustrating th...,Task 11\n\n\n\nHere is a video illustrating th...,Task 12\n\n\n\nHere is a video illustrating th...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a video illustrating thi...,Task 8\n\n\n\nHere is a video illustrating thi...,Task 9\n\n\n\nHere is a video illustrating thi...,Task 10\n\n\n\nHere is a video illustrating th...,Task 11\n\n\n\nHere is a video illustrating th...,Task 12\n\n\n\nHere is a video illustrating th...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a vid

In [ ]:
gpt2 = 0
demo = 0
no_demo = 0
count = 0
num = 0
gpt2_csv = {'Task 1': [], 'Task 2': [], 'Task 3': [], 'Task 4': [], 'Task 5': [], 'Task 6': [], 'Task 7': [], 'Task 8': [], 'Task 9': [],
            'Task 10': [], 'Task 11': [], 'Task 12': [], 'Task 13': [], 'Task 14': [], 'Task 15': [], 'Task 16': [], 'Task 17': []}
demo_csv = {'Task 1': [], 'Task 2': [], 'Task 3': [], 'Task 4': [], 'Task 5': [], 'Task 6': [], 'Task 7': [], 'Task 8': [], 'Task 9': [],
            'Task 10': [], 'Task 11': [], 'Task 12': [], 'Task 13': [], 'Task 14': [], 'Task 15': [], 'Task 16': [], 'Task 17': []}
no_demo_csv = {'Task 1': [], 'Task 2': [], 'Task 3': [], 'Task 4': [], 'Task 5': [], 'Task 6': [], 'Task 7': [], 'Task 8': [], 'Task 9': [],
            'Task 10': [], 'Task 11': [], 'Task 12': [], 'Task 13': [], 'Task 14': [], 'Task 15': [], 'Task 16': [], 'Task 17': []}
for i in range(14, len(results_csv)):
  responses_series = results_csv.loc[i][results_csv.loc[i].notnull()]
  index = (int(responses_series.keys()[0][3:]) - 32) / 17
  responses = responses_series.values[:-1]
  if len(responses) != 17:
    print(responses_series.keys()[0])
    print(responses)
    print(i)
  if i == 19:
    for j in range(11):
      sents = set([most_likely[j], all_matches[j][index], all_matches_all[j][index]])
      if len(sents) == 1:
        sents = set([most_likely[j], all_matches[j][index-1], all_matches_all[j][index-1]])
        if len(sents) == 1:
          if responses[j] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches[j][index-2]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches_all[j][index-2]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-2] and responses[j] == all_matches_all[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-2] and responses[j] == all_matches[j][index-2]:
            num += 1
          if most_likely[j] == all_matches[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] or most_likely[j] == all_matches_all[j][index-2]:
            count += 1
        else:
          if responses[j] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches[j][index-1]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches_all[j][index-1]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-1] and responses[j] == all_matches_all[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-1] and responses[j] == all_matches[j][index-1]:
            num += 1
          if most_likely[j] == all_matches[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] or most_likely[j] == all_matches_all[j][index-1]:
            count += 1
      else:
        if responses[j] == most_likely[j]:
          gpt2 += 1
          gpt2_csv['Task ' + str(j+1)].append(1)
        else:
          gpt2_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches[j][index]:
          demo += 1
          demo_csv['Task ' + str(j+1)].append(1)
        else:
          demo_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches_all[j][index]:
          no_demo += 1
          no_demo_csv['Task ' + str(j+1)].append(1)
        else:
          no_demo_csv['Task ' + str(j+1)].append(0)
        if most_likely[j] == all_matches[j][index] and responses[j] == all_matches_all[j][index] or all_matches[j][index] == all_matches_all[j][index] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index] and responses[j] == all_matches[j][index]:
          num += 1
        if most_likely[j] == all_matches[j][index] or all_matches[j][index] == all_matches_all[j][index] or most_likely[j] == all_matches_all[j][index]:
          count += 1
    gpt2_csv['Task 12'].append(0)
    demo_csv['Task 12'].append(0)
    no_demo_csv['Task 12'].append(0)
    for j in range(12, len(all_matches)):
      sents = set([most_likely[j], all_matches[j][index], all_matches_all[j][index]])
      if len(sents) == 1:
        sents = set([most_likely[j], all_matches[j][index-1], all_matches_all[j][index-1]])
        if len(sents) == 1:
          if responses[j-1] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j-1] == all_matches[j][index-2]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j-1] == all_matches_all[j][index-2]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-2] and responses[j-1] == all_matches_all[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] and responses[j-1] == most_likely[j] or most_likely[j] == all_matches_all[j][index-2] and responses[j-1] == all_matches[j][index-2]:
            num += 1
          if most_likely[j] == all_matches[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] or most_likely[j] == all_matches_all[j][index-2]:
            count += 1
        else:
          if responses[j-1] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j-1] == all_matches[j][index-1]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j-1] == all_matches_all[j][index-1]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-1] and responses[j-1] == all_matches_all[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] and responses[j-1] == most_likely[j] or most_likely[j] == all_matches_all[j][index-1] and responses[j-1] == all_matches[j][index-1]:
            num += 1
          if most_likely[j] == all_matches[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] or most_likely[j] == all_matches_all[j][index-1]:
            count += 1
      else:
        if responses[j-1] == most_likely[j]:
          gpt2 += 1
          gpt2_csv['Task ' + str(j+1)].append(1)
        else:
          gpt2_csv['Task ' + str(j+1)].append(0)
        if responses[j-1] == all_matches[j][index]:
          demo += 1
          demo_csv['Task ' + str(j+1)].append(1)
        else:
          demo_csv['Task ' + str(j+1)].append(0)
        if responses[j-1] == all_matches_all[j][index]:
          no_demo += 1
          no_demo_csv['Task ' + str(j+1)].append(1)
        else:
          no_demo_csv['Task ' + str(j+1)].append(0)
        if most_likely[j] == all_matches[j][index] and responses[j-1] == all_matches_all[j][index] or all_matches[j][index] == all_matches_all[j][index] and responses[j-1] == most_likely[j] or most_likely[j] == all_matches_all[j][index] and responses[j-1] == all_matches[j][index]:
          num += 1
        if most_likely[j] == all_matches[j][index] or all_matches[j][index] == all_matches_all[j][index] or most_likely[j] == all_matches_all[j][index]:
          count += 1
  elif i == 38:
    for j in range(4):
      sents = set([most_likely[j], all_matches[j][index], all_matches_all[j][index]])
      if len(sents) == 1:
        sents = set([most_likely[j], all_matches[j][index-1], all_matches_all[j][index-1]])
        if len(sents) == 1:
          if responses[j] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches[j][index-2]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches_all[j][index-2]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-2] and responses[j] == all_matches_all[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-2] and responses[j] == all_matches[j][index-2]:
            num += 1
          if most_likely[j] == all_matches[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] or most_likely[j] == all_matches_all[j][index-2]:
            count += 1
        else:
          if responses[j] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches[j][index-1]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches_all[j][index-1]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-1] and responses[j] == all_matches_all[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-1] and responses[j] == all_matches[j][index-1]:
            num += 1
          if most_likely[j] == all_matches[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] or most_likely[j] == all_matches_all[j][index-1]:
            count += 1
      else:
        if responses[j] == most_likely[j]:
          gpt2 += 1
          gpt2_csv['Task ' + str(j+1)].append(1)
        else:
          gpt2_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches[j][index]:
          demo += 1
          demo_csv['Task ' + str(j+1)].append(1)
        else:
          demo_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches_all[j][index]:
          no_demo += 1
          no_demo_csv['Task ' + str(j+1)].append(1)
        else:
          no_demo_csv['Task ' + str(j+1)].append(0)
        if most_likely[j] == all_matches[j][index] and responses[j] == all_matches_all[j][index] or all_matches[j][index] == all_matches_all[j][index] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index] and responses[j] == all_matches[j][index]:
          num += 1
        if most_likely[j] == all_matches[j][index] or all_matches[j][index] == all_matches_all[j][index] or most_likely[j] == all_matches_all[j][index]:
          count += 1
    gpt2_csv['Task 5'].append(0)
    demo_csv['Task 5'].append(0)
    no_demo_csv['Task 5'].append(0)
    gpt2_csv['Task 6'].append(0)
    demo_csv['Task 6'].append(0)
    no_demo_csv['Task 6'].append(0)
    gpt2_csv['Task 7'].append(0)
    demo_csv['Task 7'].append(0)
    no_demo_csv['Task 7'].append(0)
    for j in range(7, len(all_matches)):
      sents = set([most_likely[j], all_matches[j][index], all_matches_all[j][index]])
      if len(sents) == 1:
        sents = set([most_likely[j], all_matches[j][index-1], all_matches_all[j][index-1]])
        if len(sents) == 1:
          if responses[j-2] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j-2] == all_matches[j][index-2]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j-2] == all_matches_all[j][index-2]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-2] and responses[j-2] == all_matches_all[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] and responses[j-2] == most_likely[j] or most_likely[j] == all_matches_all[j][index-2] and responses[j-2] == all_matches[j][index-2]:
            num += 1
          if most_likely[j] == all_matches[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] or most_likely[j] == all_matches_all[j][index-2]:
            count += 1
        else:
          if responses[j-2] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j-2] == all_matches[j][index-1]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j-2] == all_matches_all[j][index-1]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-1] and responses[j-2] == all_matches_all[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] and responses[j-2] == most_likely[j] or most_likely[j] == all_matches_all[j][index-1] and responses[j-2] == all_matches[j][index-1]:
            num += 1
          if most_likely[j] == all_matches[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] or most_likely[j] == all_matches_all[j][index-1]:
            count += 1
      else:
        if responses[j-2] == most_likely[j]:
          gpt2 += 1
          gpt2_csv['Task ' + str(j+1)].append(1)
        else:
          gpt2_csv['Task ' + str(j+1)].append(0)
        if responses[j-2] == all_matches[j][index]:
          demo += 1
          demo_csv['Task ' + str(j+1)].append(1)
        else:
          demo_csv['Task ' + str(j+1)].append(0)
        if responses[j-2] == all_matches_all[j][index]:
          no_demo += 1
          no_demo_csv['Task ' + str(j+1)].append(1)
        else:
          no_demo_csv['Task ' + str(j+1)].append(0)
        if most_likely[j] == all_matches[j][index] and responses[j-2] == all_matches_all[j][index] or all_matches[j][index] == all_matches_all[j][index] and responses[j-2] == most_likely[j] or most_likely[j] == all_matches_all[j][index] and responses[j-2] == all_matches[j][index]:
          num += 1
        if most_likely[j] == all_matches[j][index] or all_matches[j][index] == all_matches_all[j][index] or most_likely[j] == all_matches_all[j][index]:
          count += 1
  else:
    for j in range(len(all_matches)):
      sents = set([most_likely[j], all_matches[j][index], all_matches_all[j][index]])
      if len(sents) == 1:
        sents = set([most_likely[j], all_matches[j][index-1], all_matches_all[j][index-1]])
        if len(sents) == 1:
          if responses[j] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches[j][index-2]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches_all[j][index-2]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-2] and responses[j] == all_matches_all[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-2] and responses[j] == all_matches[j][index-2]:
            num += 1
          if most_likely[j] == all_matches[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] or most_likely[j] == all_matches_all[j][index-2]:
            count += 1
        else:
          if responses[j] == most_likely[j]:
            gpt2 += 1
            gpt2_csv['Task ' + str(j+1)].append(1)
          else:
            gpt2_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches[j][index-1]:
            demo += 1
            demo_csv['Task ' + str(j+1)].append(1)
          else:
            demo_csv['Task ' + str(j+1)].append(0)
          if responses[j] == all_matches_all[j][index-1]:
            no_demo += 1
            no_demo_csv['Task ' + str(j+1)].append(1)
          else:
            no_demo_csv['Task ' + str(j+1)].append(0)
          if most_likely[j] == all_matches[j][index-1] and responses[j] == all_matches_all[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-1] and responses[j] == all_matches[j][index-1]:
            num += 1
          if most_likely[j] == all_matches[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] or most_likely[j] == all_matches_all[j][index-1]:
            count += 1
      else:
        if responses[j] == most_likely[j]:
          gpt2 += 1
          gpt2_csv['Task ' + str(j+1)].append(1)
        else:
          gpt2_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches[j][index]:
          demo += 1
          demo_csv['Task ' + str(j+1)].append(1)
        else:
          demo_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches_all[j][index]:
          no_demo += 1
          no_demo_csv['Task ' + str(j+1)].append(1)
        else:
          no_demo_csv['Task ' + str(j+1)].append(0)
        if most_likely[j] == all_matches[j][index] and responses[j] == all_matches_all[j][index] or all_matches[j][index] == all_matches_all[j][index] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index] and responses[j] == all_matches[j][index]:
          num += 1
        if most_likely[j] == all_matches[j][index] or all_matches[j][index] == all_matches_all[j][index] or most_likely[j] == all_matches_all[j][index]:
          count += 1
count

QID287
['go to a green ball' 'pick up the red ball'
 'go to the grey ball, then open the blue door and go to the red box'
 'go to the blue box, then go to the yellow ball and go to a purple key'
 'pick up the green box, then go to a blue key, then put a green box next to a green ball'
 'go to a blue ball' 'open the green door and go to the red ball'
 'open a blue door, then pick up the blue key'
 'open the blue door and go to the blue ball'
 'open the red door, then open the red door'
 'open the purple door, then go to the green ball, then go to the red box'
 'open the blue door and go to the blue ball'
 'open the green door and go to the green door'
 'open the red door and go to a red key, then open a blue door'
 'open a green door and open a green door and go to a green door'
 'open the red door, then open a green door']
19
QID1018
['go to a blue ball'
 'pick up a red ball, then put the red ball next to a yellow box'
 'open a blue door and open a purple door and go to a red box'
 'go

294

In [ ]:
results2_csv = pd.read_csv('drive/MyDrive/Baby AI Evaluation Take All 2.csv')
results2_csv

,QID32,QID33,QID34,QID35,QID36,QID37,QID38,QID39,QID40,QID41,QID42,QID43,QID44,QID45,QID46,QID47,QID48,QID49,QID50,QID51,QID52,QID53,QID54,QID55,QID56,QID57,QID58,QID59,QID60,QID61,QID62,QID63,QID64,QID65,QID66,QID67,QID68,QID69,QID70,QID71,...,QID673,QID674,QID675,QID676,QID677,QID678,QID679,QID680,QID681,QID682,QID683,QID684,QID685,QID686,QID687,QID688,QID689,QID690,QID691,QID692,QID693,QID694,QID695,QID696,QID697,QID698,QID699,QID700,QID701,QID702,QID703,QID704,QID705,QID706,QID707,QID708,QID709,QID710,QID711,Random ID
0,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a video illustrating thi...,Task 8\n\n\n\nHere is a video illustrating thi...,Task 9\n\n\n\nHere is a video illustrating thi...,Task 10\n\n\n\nHere is a video illustrating th...,Task 11\n\n\n\nHere is a video illustrating th...,Task 12\n\n\n\nHere is a video illustrating th...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a video illustrating thi...,Task 8\n\n\n\nHere is a video illustrating thi...,Task 9\n\n\n\nHere is a video illustrating thi...,Task 10\n\n\n\nHere is a video illustrating th...,Task 11\n\n\n\nHere is a video illustrating th...,Task 12\n\n\n\nHere is a video illustrating th...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a video illustrating thi...,Task 8\n\n\n\nHere is a video illustrating thi...,Task 9\n\n\n\nHere is a video illustrating thi...,Task 10\n\n\n\nHere is a video illustrating th...,Task 11\n\n\n\nHere is a video illustrating th...,Task 12\n\n\n\nHere is a video illustrating th...,Task 13\n\n\n\nHere is a video illustrating th...,Task 14\n\n\n\nHere is a video illustrating th...,Task 15\n\n\n\nHere is a video illustrating th...,Task 16\n\n\n\nHere is a video illustrating th...,Task 17\n\n\n\nHere is a video illustrating th...,Task 1\n\n\n\nHere is a video illustrating thi...,Task 2\n\n\n\nHere is a video illustrating thi...,Task 3\n\n\n\nHere is a video illustrating thi...,Task 4\n\n\n\nHere is a video illustrating thi...,Task 5\n\n\n\nHere is a video illustrating thi...,Task 6\n\n\n\nHere is a video illustrating thi...,Task 7\n\n\n\nHere is a video illustrating thi...,Task 8\n\n\n\nHe

In [ ]:
len(results_csv) - 14 + len(results2_csv) - 9

50

In [ ]:
for i in range(9, len(results2_csv)):
  responses_series = results2_csv.loc[i][results2_csv.loc[i].notnull()]
  index = (int(responses_series.keys()[0][3:]) - 32) / 17 + 87
  responses = responses_series.values[:-1]
  if len(responses) != 17:
    print('oh no')
  for j in range(len(all_matches)):
    sents = set([most_likely[j], all_matches[j][index], all_matches_all[j][index]])
    if len(sents) == 1:
      sents = set([most_likely[j], all_matches[j][index-1], all_matches_all[j][index-1]])
      if len(sents) == 1:
        if responses[j] == most_likely[j]:
          gpt2 += 1
          gpt2_csv['Task ' + str(j+1)].append(1)
        else:
          gpt2_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches[j][index-2]:
          demo += 1
          demo_csv['Task ' + str(j+1)].append(1)
        else:
          demo_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches_all[j][index-2]:
          no_demo += 1
          no_demo_csv['Task ' + str(j+1)].append(1)
        else:
          no_demo_csv['Task ' + str(j+1)].append(0)
        if most_likely[j] == all_matches[j][index-2] and responses[j] == all_matches_all[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-2] and responses[j] == all_matches[j][index-2]:
          num += 1
        if most_likely[j] == all_matches[j][index-2] or all_matches[j][index-2] == all_matches_all[j][index-2] or most_likely[j] == all_matches_all[j][index-2]:
          count += 1
      else:
        if responses[j] == most_likely[j]:
          gpt2 += 1
          gpt2_csv['Task ' + str(j+1)].append(1)
        else:
          gpt2_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches[j][index-1]:
          demo += 1
          demo_csv['Task ' + str(j+1)].append(1)
        else:
          demo_csv['Task ' + str(j+1)].append(0)
        if responses[j] == all_matches_all[j][index-1]:
          no_demo += 1
          no_demo_csv['Task ' + str(j+1)].append(1)
        else:
          no_demo_csv['Task ' + str(j+1)].append(0)
        if most_likely[j] == all_matches[j][index-1] and responses[j] == all_matches_all[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index-1] and responses[j] == all_matches[j][index-1]:
          num += 1
        if most_likely[j] == all_matches[j][index-1] or all_matches[j][index-1] == all_matches_all[j][index-1] or most_likely[j] == all_matches_all[j][index-1]:
          count += 1
    else:
      if responses[j] == most_likely[j]:
        gpt2 += 1
        gpt2_csv['Task ' + str(j+1)].append(1)
      else:
        gpt2_csv['Task ' + str(j+1)].append(0)
      if responses[j] == all_matches[j][index]:
        demo += 1
        demo_csv['Task ' + str(j+1)].append(1)
      else:
        demo_csv['Task ' + str(j+1)].append(0)
      if responses[j] == all_matches_all[j][index]:
        no_demo += 1
        no_demo_csv['Task ' + str(j+1)].append(1)
      else:
        no_demo_csv['Task ' + str(j+1)].append(0)
      if most_likely[j] == all_matches[j][index] and responses[j] == all_matches_all[j][index] or all_matches[j][index] == all_matches_all[j][index] and responses[j] == most_likely[j] or most_likely[j] == all_matches_all[j][index] and responses[j] == all_matches[j][index]:
        num += 1
      if most_likely[j] == all_matches[j][index] or all_matches[j][index] == all_matches_all[j][index] or most_likely[j] == all_matches_all[j][index]:
        count += 1
count

389

In [ ]:
num

119

In [ ]:
gpt2 + demo + no_demo

1116

In [ ]:
gpt2

274

In [ ]:
gpt2 / 1116

0.24551971326164876

In [ ]:
demo

462

In [ ]:
demo / 1116

0.41397849462365593

In [ ]:
no_demo

380

In [ ]:
no_demo / 1116

0.34050179211469533

In [ ]:
for i in range(1, 18):
  for j in range(16):
    gpt2_csv['Task ' + str(i)].append(0)
gpt2_df = pd.DataFrame.from_dict(gpt2_csv)
gpt2_df.to_csv('drive/MyDrive/GPT-2 Full.csv', index=False)

In [ ]:
for i in range(1, 18):
  for j in range(16):
    demo_csv['Task ' + str(i)].append(0)
demo_df = pd.DataFrame.from_dict(demo_csv)
demo_df.to_csv('drive/MyDrive/Demo Full.csv', index=False)

In [ ]:
for i in range(1, 18):
  for j in range(16):
    no_demo_csv['Task ' + str(i)].append(0)
no_demo_df = pd.DataFrame.from_dict(no_demo_csv)
no_demo_df.to_csv('drive/MyDrive/No Demo Full.csv', index=False)

# Explanation

In [ ]:
model = SentenceTransformer('stsb-distilbert-base')

In [ ]:
sentences_for_each_task = [pickle.load(open('drive/MyDrive/babyai_sentences' + str(i) + '.pkl', 'rb')) for i in range(17)]
#embeddings_for_each_task = [model.encode(sentences_for_each_task[i], convert_to_tensor=True).cpu() for i in range(len(sentences_for_each_task))]
#pickle.dump(embeddings_for_each_task, open('drive/MyDrive/embeddings_for_each_task_cpu.pkl', 'wb'))

In [ ]:
sentences_for_all_tasks = [sentence for l in sentences_for_each_task for sentence in l]
len(sentences_for_all_tasks)

195401

In [ ]:
embeddings_for_all_tasks = model.encode(sentences_for_all_tasks, convert_to_tensor=True)
len(embeddings_for_all_tasks)

Exception ignored in: <generator object tqdm_notebook.__iter__ at 0x7f37a3857ed0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tqdm/notebook.py", line 220, in __iter__
    self.sp(bar_style='danger')
AttributeError: 'tqdm_notebook' object has no attribute 'sp'


KeyboardInterrupt: ignored

In [ ]:
pickle.dump(embeddings_for_all_tasks_cpu, open('drive/MyDrive/embeddings_for_all_tasks_cpu.pkl', 'wb'))

In [ ]:
embeddings_for_each_task = pickle.load(open('drive/MyDrive/embeddings_for_each_task_cpu.pkl', 'rb'))
#embeddings_for_each_task_cpu = [tensor.cpu() for tensor in embeddings_for_each_task]
len(embeddings_for_each_task)

17

In [ ]:
pickle.dump(embeddings_for_each_task_cpu, open('drive/MyDrive/embeddings_for_each_task_cpu.pkl', 'wb'))

In [ ]:
embeddings_for_all_tasks = pickle.load(open('drive/MyDrive/embeddings_for_all_tasks_cpu.pkl', 'rb'))
#embeddings_for_all_tasks_cpu = embeddings_for_all_tasks.cpu()
len(embeddings_for_all_tasks)

195401

In [ ]:
def explanation(task_id, user_sentence, variant):
  # GPT-2
  if variant == 0:
    return most_likely[task_id]
  else:
    for punct in punctuation:
        user_sentence = user_sentence.replace(punct, '')
    user_sentence = user_sentence.lower()
    embedding = model.encode(user_sentence, convert_to_tensor=True)
    # No-demo
    if variant == 1:
      cosine_scores = util.pytorch_cos_sim(embedding, embeddings_for_all_tasks)
      closest = sentences_for_all_tasks[cosine_scores.argmax(1)]
    # Our tool
    else:
      cosine_scores = util.pytorch_cos_sim(embedding, embeddings_for_each_task[task_id])
      closest = sentences_for_each_task[task_id][cosine_scores.argmax(1)]
    return closest

In [ ]:
explanation(16, 'go to the magnolia ball', 1)

'go to the blue ball and pick up a grey ball'